In [1]:
import sys
import time
import os
from ctypes import *

def kdc():
    def measurement_cycle(serial_num, a):
        # set a new position in device units
        new_pos_real = c_double(-(a - 25.0))  # in real units (in mm)
        new_pos_dev = c_int()
        lib.CC_GetDeviceUnitFromRealValue(serial_num,
                                          new_pos_real,
                                          byref(new_pos_dev),
                                          0)

        print(f"The device has moved a translation of {a} mm")
        print(f'{new_pos_real.value} mm in Device Units: {new_pos_dev.value}')

        # Move to new position as an absolute move.
        lib.CC_SetMoveAbsolutePosition(serial_num, new_pos_dev)
        time.sleep(0.25)
        lib.CC_MoveAbsolute(serial_num)

    if sys.version_info < (3, 8):
        os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
    else:
        os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")

    lib: CDLL = cdll.LoadLibrary("Thorlabs.MotionControl.KCube.DCServo.dll")

    # Uncomment this line if you are using simulations
    # lib.TLI_InitializeSimulations()

    # Set constants
    serial_num = c_char_p(b"27005881")

    # Open the device
    if lib.TLI_BuildDeviceList() == 0:
        lib.CC_Open(serial_num)
        lib.CC_StartPolling(serial_num, c_int(200))

        # Home the device
        lib.CC_Home(serial_num)
        time.sleep(1)

        # Set up the device to convert real units to device units
        STEPS_PER_REV = c_double(34555)  # for the PRM1-Z8
        gbox_ratio = c_double(1.0)  # gearbox ratio
        pitch = c_double(1.0)

        # Apply these values to the device
        lib.CC_SetMotorParamsExt(serial_num, STEPS_PER_REV, gbox_ratio, pitch)

        # Get the device's current position in dev units
        lib.CC_RequestPosition(serial_num)
        time.sleep(0.2)
        dev_pos = c_int(lib.CC_GetPosition(serial_num))

        # Convert device units to real units
        real_pos = c_double()
        lib.CC_GetRealValueFromDeviceUnit(serial_num,
                                          real_pos,
                                          byref(real_pos),
                                          0)

        print(f'Position after homing: {real_pos.value}')

        # Get user input for final translation 'b' and interval 'c'
        b = float(input("Enter the final translation 'b' (should be less than or equal to 25): "))
        c = float(input("Enter the interval 'c': "))
        pause_duration = float(input("Enter the pause duration (in seconds): "))

        # Ensure 'b' is less than or equal to 25
        if b > 25:
            print("Error: 'b' should be less than or equal to 25.")
        else:
            # Perform the measurement cycle for values of 'a' at intervals 'c' until 'b'
            a = 0
            while a <= b:
                measurement_cycle(serial_num, a)
                time.sleep(pause_duration)
                a += c

            # Go back to the original location at 'a = 0'
            measurement_cycle(serial_num, 0)
            time.sleep(pause_duration)

        # Close the device
        lib.CC_Close(serial_num)
        # lib.TLI_UninitializeSimulations()

    return

    if __name__ == "__main__":
        main()

In [ ]:
kdc()

Position after homing: 0.0


Enter the final translation 'b' (should be less than or equal to 25):  10
Enter the interval 'c':  0
Enter the pause duration (in seconds):  0


The device has moved a translation of 0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
The device has moved a translation of 0.0 mm
25.0 mm in Device Units: 863875
T